In [ ]:
!pip install PyMuPDF
!pip install re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 28.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [ ]:
from pdfminer.high_level import extract_text
import re

def extract_titles_and_headings(pdf_path):
    text = extract_text(pdf_path)

    # Define regex patterns to identify titles, headings, and sections
    title_pattern = re.compile(r'((?<=\n)\s*[A-Z].+?(?=\n))')
    heading_pattern = re.compile(r'((?<=\n)\s*[A-Z].+?(?=\n))')
    section_pattern = re.compile(r'((?<=\n)\s*[0-9]+\s*Section.+?(?=\n))')

    titles = title_pattern.findall(text)
    headings = heading_pattern.findall(text)
    sections = section_pattern.findall(text)

    return titles, headings, sections

def main():
    pdf_path = '/content/civil-code.pdf'
    titles, headings, sections = extract_titles_and_headings(pdf_path)

    print("Titles:")
    for title in titles:
        print(title)

    print("\nHeadings:")
    for heading in headings:
        print(heading)

    print("\nSections:")
    for section in sections:
        print(section)

if __name__ == "__main__":
    main()


In [61]:
import fitz
import re

In [ ]:
def extract_sentences_with_starting_words(pdf_path, starting_words):
    doc = fitz.open(pdf_path)
    word_sentences = {word: [] for word in starting_words}
    current_word = None
    previous_line_empty = False
    previous_line_word = None

    for page in doc:
        text = page.get_text()
        lines = text.splitlines()
        for line in lines:
            if line.strip():  # Check if the line is not empty
                # Check if the line starts with any of the starting words
                for word in starting_words:
                    if line.lower().startswith(word.lower()):
                        current_word = word
                        word_sentences[current_word].append(line.strip())
                        previous_line_word = word
                        break
                else:
                    # Append the line to the current sentence if a starting word is detected in the previous line
                    if current_word:
                        word_sentences[current_word][-1] += ' ' + line.strip()
                        previous_line_word = current_word
            elif previous_line_word and not previous_line_empty:
                # End the sentence if the current line is empty and the previous line wasn't empty
                current_word = None
                previous_line_word = None

            previous_line_empty = not line.strip()  # Update the flag for the previous line

    doc.close()
    return word_sentences


In [63]:
pdf_path = "/content/civil-code.pdf"
starting_words = ["Titre", "Chapitre", "Section", "Sous-section", "Article"]
# Extract sentences with specified starting words
word_sentences = extract_sentences_with_starting_words(pdf_path, starting_words)

# Print the sentences for each starting word
for word, sentences in word_sentences.items():
    print(f"{word} Sentences:")
    for sentence in sentences:
        print(sentence)
    print()

Titre Sentences:
Titre préliminaire : De la publication, des effets et de l'application des lois en général
Titre Ier : Des droits civils
Titre Ier bis : De la nationalité française
titre de l'infraction définie à l'article 222-9 du code pénal, lorsque celle-ci a été commise sur un mineur de quinze ans, sont constitutives du défaut d'assimilation.
Titre II : Des actes de l'état civil
Titre III : Du domicile
Titre IV : Des absents
Titre V : Du mariage
Titre VI : Du divorce
Titre VII : De la filiation
titre a duré au moins cinq ans depuis la naissance ou la reconnaissance, si elle a été faite ultérieurement.
Titre VIII : De la filiation adoptive
Titre IX : De l'autorité parentale
titre VII du présent livre, lorsque la mention de la reconnaissance conjointe est apposée à la demande du procureur de la République.
titre VIII du livre V du code de la sécurité sociale et par le code de procédure civile.
Titre X : De la minorité, de la tutelle et de l'émancipation
Titre XI : De la majorité et 

In [ ]:
def extract_sentences_with_starting_words(pdf_path, starting_words):
    doc = fitz.open(pdf_path)
    word_sentences = {word: [] for word in starting_words}
    current_word = None

    for page in doc:
        text = page.get_text()
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\n)\s', text)
        for sentence in sentences:
            for word in starting_words:
                if sentence.lower().startswith(word.lower()):
                    current_word = word
                    word_sentences[current_word].append(sentence.strip())
                elif current_word and sentence.strip().startswith(' '):
                    word_sentences[current_word][-1] += ' ' + sentence.strip()

    doc.close()
    return word_sentences

In [ ]:
def extract_sections(pdf_path):
    doc = fitz.open(pdf_path)
    sections = {}

    current_section = None
    current_content = []

    for page in doc:
        text = page.get_text()
        lines = text.split("\n")
        for line in lines:
            line = line.strip()
            if line:
                # Check for section headings
                if re.match(r'^\s*(Chapitre|Section|Article)\s+\d+', line):
                    # If a new section heading is found, save the previous section content
                    if current_section:
                        sections[current_section] = "\n".join(current_content)
                        current_content = []
                    current_section = line
                else:
                    # Add content to the current section
                    if current_section:
                        current_content.append(line)

    # Save the content of the last section
    if current_section:
        sections[current_section] = "\n".join(current_content)

    doc.close()
    return sections

In [ ]:
# PDF path
pdf_path = "/content/civil-code.pdf"

# Extract sections from the PDF
sections = extract_sections(pdf_path)

# Print the sections
for section, content in sections.items():
    print(f"{section}:\n{content}\n")

Streaming output truncated to the last 5000 lines.

Article 1672:
Si l'acquéreur a laissé plusieurs héritiers, l'action en rachat ne peut être exercée contre chacun d'eux que pour
sa part, dans le cas où elle est encore indivise, et dans celui où la chose vendue a été partagée entre eux.
Mais s'il y a eu partage de la succession et que la chose vendue soit échue au lot de l'un des héritiers, l'action
en rachat peut être intentée contre lui pour le tout.

Article 1673:
Le vendeur qui use du pacte de rachat doit rembourser non seulement le prix principal, mais encore les
frais et loyaux coûts de la vente, les réparations nécessaires, et celles qui ont augmenté la valeur du fonds,
jusqu'à concurrence de cette augmentation. Il ne peut entrer en possession qu'après avoir satisfait à toutes ces
obligations.
Lorsque le vendeur rentre dans son héritage par l'effet du pacte de rachat, il le reprend, exempt de toutes
les charges et hypothèques dont l'acquéreur l'aurait grevé, à la condition que 

In [ ]:
def extract_sentences_with_keywords(pdf_path, keywords):
    doc = fitz.open(pdf_path)
    keyword_sentences = {keyword: [] for keyword in keywords}

    for page in doc:
        text = page.get_text()
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\n)\s{3,}', text)
        for sentence in sentences:
            for keyword in keywords:
                if re.match(r'^\s*(.*?\b{}\b.*?)\b'.format(keyword.lower()), sentence.lower()):
                    keyword_sentences[keyword].append(sentence.strip())

    doc.close()
    return keyword_sentences

In [ ]:
pdf_path = "/content/civil-code.pdf"
keywords = ["Chapitre"]

# Extract sentences with specified keywords
keyword_sentences = extract_sentences_with_keywords(pdf_path, keywords)

# Print the sentences for each keyword
for keyword, sentences in keyword_sentences.items():

    print(f"{keyword} Sentences:")
    for sentence in sentences:
        print(sentence)
    print()

Chapitre Sentences:
Chapitre II : Du respect du corps humain
Article 16
 
La loi assure la primauté de la personne, interdit toute atteinte à la dignité de celle-ci et garantit le respect de
l'être humain dès le commencement de sa vie.
Chapitre II : De la nationalité française d'origine
Section 1 : Des Français par filiation
Article 18
 
Est français l'enfant dont l'un des parents au moins est français.
Chapitre III : De l'acquisition de la nationalité française
Section 1 : Des modes d'acquisition de la nationalité française
Paragraphe 1 : Acquisition de la nationalité française à raison de la
filiation
Article 21
 
L'adoption simple n'exerce de plein droit aucun effet sur la nationalité de l'adopté.
Chapitre IV : De la perte, de la déchéance et de la réintégration dans
la nationalité française
2° S'il est condamné pour un acte qualifié de crime ou délit prévu et réprimé par le chapitre II du titre III du
livre IV du code pénal ;
Chapitre V : Des actes relatifs à l'acquisition ou à la 